# Unsupervised Machine Learning GEE

Unsupervised classification of Landsat-8 image using Kmeans clustering.

### libraries

In [2]:
import ee
import geemap

### Interactive Map

In [3]:
Map = geemap.Map()
Map.setCenter(36.8, -1.3, 11)
Map

Map(center=[-1.3, 36.8], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=…

### Add data to map

In [4]:
point = ee.Geometry.Point([36.9, -1.4])

image = (
    ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
    .filterBounds(point)
    .filterDate('2020-01-01', '2020-12-31')
    .sort('CLOUD_COVER')
    .first()
    .select('B[1-7]')
)

vis_params = {
    'min': 0,
    'max': 3000,
    'bands': ['B5', 'B4', 'B3']
}

Map.addLayer(image, vis_params, "Landsat-8")

### Check image properties

In [5]:
props = geemap.image_props(image)
props.getInfo()

{'CLOUD_COVER': 1.3,
 'CLOUD_COVER_LAND': 1.3,
 'EARTH_SUN_DISTANCE': 0.988637,
 'ESPA_VERSION': '2_23_0_1b',
 'GEOMETRIC_RMSE_MODEL': 7.564,
 'GEOMETRIC_RMSE_MODEL_X': 5.101,
 'GEOMETRIC_RMSE_MODEL_Y': 5.586,
 'IMAGE_DATE': '2020-02-20',
 'IMAGE_QUALITY_OLI': 9,
 'IMAGE_QUALITY_TIRS': 9,
 'LANDSAT_ID': 'LC08_L1TP_168061_20200220_20200225_01_T1',
 'LEVEL1_PRODUCTION_DATE': 1582645024000,
 'NOMINAL_SCALE': 30,
 'PIXEL_QA_VERSION': 'generate_pixel_qa_1.6.0',
 'SATELLITE': 'LANDSAT_8',
 'SENSING_TIME': '2020-02-20T07:43:11.8975180Z',
 'SOLAR_AZIMUTH_ANGLE': 108.893326,
 'SOLAR_ZENITH_ANGLE': 31.808987,
 'SR_APP_VERSION': 'LaSRC_1.3.0',
 'WRS_PATH': 168,
 'WRS_ROW': 61,
 'system:asset_size': '619.541719 MB',
 'system:band_names': ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7'],
 'system:id': 'LANDSAT/LC08/C01/T1_SR/LC08_168061_20200220',
 'system:index': 'LC08_168061_20200220',
 'system:time_end': '2020-02-20 07:43:11',
 'system:time_start': '2020-02-20 07:43:11',
 'system:version': 15829636934

### Make trainiing dataset

Create a region for generating training dataset.
1. Draw a shape.
2. Define Geometry
3. Create a buffer zone from a point
4. If no region is defined, it will use image footprint by default

In [6]:
#regions
region = ee.Geometry.Point(point.buffer(10000))

In [7]:
training = image.sample(
  **{
        #     'region': region,
        'scale': 30,
        'numPixels': 5000,
        'seed': 0,
        'geometries': True,  # Set this to False to ignore geometries
    }  
)

Map.addLayer(training, {}, 'training', False)
Map

Map(center=[-1.3, 36.8], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=…

#### Train the clusterer

In [8]:
#create cluster instance and train it
num_clusters = 5
clusterer = ee.Clusterer.wekaKMeans(num_clusters).train(training)

In [9]:
#cluster the input using trained clusterer
result = image.cluster(clusterer)

#visualize the clusters with random colors.
Map.addLayer(result.randomVisualizer(), {}, 'clusters')
Map

Map(center=[-1.3, 36.8], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=…

### Label clusters

In [10]:
legend_keys = ['One', 'Two', 'Three', 'Four', 'ect']
legend_colors = ['#8DD3C7', '#FFFFB3', '#BEBADA', '#FB8072', '#80B1D3']

#Reclassify map
result = result.remap([0, 1, 2, 3, 4], [1, 2, 3, 4, 5])

#legend colors
Map.addLayer(
    result, {'min': 1, 'max': 5, 'palette': legend_colors}, 'Labelled clusters'
)
#add legend
Map.add_legend(
    legend_keys=legend_keys, legend_colors=legend_colors, position='bottomright'
)
Map

Map(center=[-1.3, 36.8], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=…

### visualize

In [11]:
print('Change layer opacity:')
cluster_layer = Map.layers[-1]
cluster_layer.interact(opacity=(0, 1, 0.1))

Change layer opacity:


Box(children=(FloatSlider(value=1.0, description='opacity', max=1.0),))

### Save to device

In [13]:
import os

out_dir = os.path.join(os.path.expanduser('~'), 'gisData/classification/gee/unsupervised')
out_file = os.path.join(out_dir, 'cluster.tif')

geemap.ee_export_image(result, filename=out_file, scale=90)

Generating URL ...
Please wait ...
Data downloaded to /home/jeff/gisData/classification/gee/unsupervised/cluster.tif
